# Train SD LoRA with kohya_ss

## Instructions

### To generate captions:
- Utilities -> Captioning -> BLIP Captioning -> choose Image folder to caption -> Min length: 20 -> Caption images

### To train LoRA:
- LoRA -> Training -> Source model -> runwayml/stable-diffusion-v1-5 -> Folders -> choose Image folder -> choose Output folder -> choose Model output name -> Parameters -> Presets: sd15-EDG_LoraOptiSettings -> Mixed precision: fp16 -> Save precision: fp16 -> Optimizer: AdamW -> Start training

## Links
- orig post: https://stable-diffusion-art.com/train-lora/
  - notebook: https://colab.research.google.com/github/sagiodev/SDA-lora-training/blob/main/LoRA_trainer_kohya_ss_SDA.ipynb
- diffusion trainer orig repo: https://github.com/kohya-ss/sd-scripts
- gui of it: https://github.com/bmaltais/kohya_ss
- colab of it: https://github.com/camenduru/kohya_ss-colab
  - notebook: https://colab.research.google.com/github/camenduru/kohya_ss-colab/blob/main/kohya_ss_colab.ipynb
- lora training tutorial: https://www.canva.com/design/DAFcn1l_ulE/view#1
- lora training parameters: https://github.com/bmaltais/kohya_ss/wiki/LoRA-training-parameters


In [ ]:
# upload images

import os
import shutil
from google.colab import files, drive
drive.mount('/content/drive')

dataset_name = 'AndyLau'
num_epochs = 100

dataset_path = '/content/drive/MyDrive/SDFiles/datasets/' + dataset_name
images_path = dataset_path + '/' + str(num_epochs) + '_' + dataset_name # bmaltais/kohya_ss requires the images directory to be inside the dataset directory, and has <int>_<text> format
output_path = '/content/drive/MyDrive/SDFiles/Lora'

!mkdir -p {output_path}

if os.path.exists(images_path):
  print(f'Error: Folder {images_path} already exists.\n')
else:
  !mkdir -p {images_path}
  uploaded = files.upload()
  for filename in uploaded.keys():
    image_path = images_path + '/' + filename
    shutil.move(filename, image_path)
  print('Images uploaded successfully.\n')

print(f"Utilities -> Captioning -> BLIP Captioning -> 'Image folder to caption': {images_path}")
# /content/drive/MyDrive/SDFiles/datasets/AndyLau/100_AndyLau
print(f"LoRA -> Training -> Folders -> 'Image folder': {dataset_path}")
# /content/drive/MyDrive/SDFiles/datasets/AndyLau
print(f"LoRA -> Training -> Folders -> 'Output folder': {output_path}")
# /content/drive/MyDrive/SDFiles/Lora

In [ ]:
# train LoRA using kohya_ss GUI

from google.colab import drive
drive.mount('/content/drive')

!pip install dadaptation==3.1 diffusers[torch]==0.17.1 easygui==0.98.3 einops==0.6.0 fairscale==0.4.13 ftfy==6.1.1 gradio==3.36.1 huggingface-hub==0.14.1
!pip install lion-pytorch==0.0.6 lycoris_lora==1.8.0.dev6 open-clip-torch==2.20.0 prodigyopt==1.0 pytorch-lightning==1.9.0 safetensors==0.3.1 timm==0.6.12
!pip install tk==0.1.0 transformers==4.30.2 voluptuous==0.13.1 wandb==0.15.0 xformers==0.0.20 omegaconf

def clear():
  from IPython.display import clear_output; return clear_output()

# install bitsandbytes
!git clone -b 0.41.0 https://github.com/TimDettmers/bitsandbytes
%cd /content/bitsandbytes
!CUDA_VERSION=118 make cuda11x
!python setup.py install

%cd /content
!git clone https://github.com/bmaltais/kohya_ss.git
%cd kohya_ss/
!git checkout v21.8.9

clear()

# add pwd to python path or else blip captioning won't work
%env PYTHONPATH=/env/python:/content/kohya_ss
!python kohya_gui.py --headless --share